# This is the code for fairness task in SIOP ML competition 2024
Author: Zihao Jia zjia2@gmu.edu


***Note: all the existing outputs in the current file were original outputs we used for submission.***

## Analysis strategy
###Step 1: Using GPT-4 to create a explanation why the first/second policy is more preferred. This is designed to provide a thought process for the GPT model for later classifications.
###Step 2: Using a few-shot learning GPT-4 model to classify which policy is more preferred. During few shot learning, instead of only providing the original policies, we are also producing the reasoning process generated from step 1 to help model create a thinking process.

In [ ]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# install pacakges
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.6 MB/s eta 0:00:00


In [ ]:
# Read the datasets, you may need to change the directory. But the .csv file names should be the same.
input_file_path = '/content/drive/MyDrive/2024 SIOP competition/Reproduce/data/fairness_train.csv'
eval_file_path = '/content/drive/MyDrive/2024 SIOP competition/Reproduce/data/fairness_val_public.csv'
test_file_path = '/content/drive/MyDrive/2024 SIOP competition/Reproduce/data/Copy of fairness_test_public.csv'

In [ ]:
import pandas as pd

# Replace 'your_file.csv' with the name of your CSV file
ds_fairness = pd.read_csv(input_file_path)

# Step 1: Generate explanations for training set data.
We have already saved and uploaded the new file with generated explanations. You are NOT required to run these codes if your purpose is to reproduce the results for competition only.

In [ ]:
# Import API for OpenAI
from openai import OpenAI
import openai
import os

client = OpenAI(api_key= 'your api')

In [ ]:
# we are generating an explanation for why first/second policy is more preferred for each case in training set.
# This step has been done with GPT 4
# We have also saved the generated reasons so you do not have to replicate this process again.
ds_fairness['reason_gpt4'] = None
for i in range(len(ds_fairness)):
  completion = client.chat.completions.create(
  model="gpt-4-0125-preview",
  temperature=1,
  messages=[{"role": "user", "content": f"I have two policies.Policy 1:[{ds_fairness.first_option[i]}].Policy 2: [{ds_fairness.second_option[i]}] employees prefer the {ds_fairness.majority_vote[i]} one. What could be the reasons for that? Provide your answer"}],
  )
  ds_fairness['reason_gpt4'][i] = completion.choices[0].message.content

0     The preference for Policy 1 (Conflict Resoluti...
1     Employees might prefer the Diversity and Inclu...
2     Employees' preference for Policy 2 over Policy...
3     Employees might prefer Policy 1 over Policy 2 ...
4     Employees might prefer Policy 1 (the formation...
5     Employees might prefer Policy 1 over Policy 2 ...
6     Employees might prefer Policy 1 (Employee Supp...
7     The preference for Policy 1: [Training in conf...
8     Employees might prefer Policy 2, which involve...
9     Employees might prefer Policy 2 over Policy 1 ...
10    Employees might prefer Policy 2 over Policy 1 ...
11    There are several reasons why employees might ...
12    Employees might prefer Policy 2, which outline...
13    Employees might prefer Policy 2, which establi...
14    Employees might prefer Policy 2, which encoura...
15    Employees' preference for Policy 2, which ackn...
16    Employees might prefer Policy 2 over Policy 1 ...
17    There could be several reasons why employe

In [ ]:
#ds_fairness.to_csv('/content/drive/MyDrive/2024 SIOP competition/data/fairness_train_reason.csv')

# Step 2: generate resutls with the updated training data.

In [ ]:
# here is the updated training data file with generate explanations.
ds_fairness = pd.read_csv('/content/drive/MyDrive/2024 SIOP competition/Reproduce/data/fairness_train_reason.csv')

In [ ]:
# read the test set
ds_eval = pd.read_csv(test_file_path)

In [ ]:
# we are aggregating all examples in the training set to serve as few-shot learning examples later.
all_prompt = []
for i in range(len(ds_fairness)):
  conversation = {"role": "user", "content": f"Here are two organizational policies, which one do you prefer? First policy: [{ds_fairness.first_option[i]}]. Second policy:[{ds_fairness.second_option[i]}]. Tell me 'first' or 'second' at the beginning of your response, then tell me why"},{"role": "assistant", "content": f" {ds_fairness.majority_vote[i]}. Because {ds_fairness.reason_gpt4[i]}, so I prefer the {ds_fairness.majority_vote[i]} one."}
  all_prompt.append(conversation)

In [ ]:
# iterate within the test set to generate responses.
# All examples in the training set will serve as few-shot learning examples
# One question is added to the end of few-shot learning examples. And that question follows the loop in our test set.
from itertools import chain
predict_666 = []
for i in range(len(ds_eval)):
  question = {"role": "user", "content": f"Here are two organizational policies, which one do you prefer? First policy:[{ds_eval.first_option[i]}].Second policy: [{ds_eval.second_option[i]}]. Only tell me first or second in this response. DO NOT show your explaination in your response."}
  all_prompt_flat = list(chain.from_iterable(all_prompt))
  all_prompt_flat.append(question)
  completion = client.chat.completions.create(
    model="gpt-4-0125-preview",
    temperature=0.8,
    messages=all_prompt_flat,
    seed=666
  )
  predict_666.append(completion.choices[0].message.content)

In [ ]:
# here are our predicted results (same results in the final submission)
predict_666

['second',
 'second',
 'first',
 'first',
 'first',
 'first',
 'first',
 'first',
 'first',
 'first',
 'first',
 'second',
 'second',
 'second',
 'first',
 'first',
 'first',
 'second',
 'second',
 'second',
 'first',
 'second',
 'second',
 'second',
 'first',
 'first',
 'first',
 'first',
 'second',
 'second',
 'first',
 'first',
 'first',
 'first',
 'first',
 'second',
 'first',
 'first',
 'second',
 'second',
 'first',
 'second',
 'first',
 'second',
 'first',
 'second',
 'second',
 'second',
 'first',
 'first',
 'second',
 'second',
 'second',
 'first',
 'second',
 'first',
 'first',
 'second']